In [ ]:
from tkinter import messagebox
import customtkinter as ctk
import tkinter as tk
import subprocess
import time
import json
import wmi
import os

#default_path = "E:\\Driver"
default_path = os.path.expanduser("~\Desktop\Driver")

font = ('Dubai Medium', 15)
font1 = ('Dubai Medium', 15, 'bold')
font2= ('Dubai Medium', 16)

driver_search_keywords = {
    "Intel® Chipset": ["chipset"],
    "Intel® Management Engine": ["management engine"],
    "Intel® Graphics": ["graphics"],
    "Intel® Serial IO": ["serial io"],
    "Intel® Network": ["network"],
    "Intel® GNA": ["gna"],
    "Intel® HID": ["hid"],
    "AcpiSmbus": ["acpismbus"],
    "KSC20": ["ksc20"],
    "Core-Temp": ["core-temp"],
    "bitwindows": ["bitwindows"],
    "FUSBCAN": ["fusbcam"]
}

def start_drag(event, window):
    window.x_offset = event.x
    window.y_offset = event.y

def on_drag(event, window):
    x = window.winfo_pointerx() - window.x_offset
    y = window.winfo_pointery() - window.y_offset
    window.geometry(f"+{x}+{y}")

def save_selected_drivers(selected_drivers):
    state = {option: var.get() for option, var in selected_drivers.items()}
    #print(state)
    save_file = "sel_drivers.json" 
    print(f"Saving sel_drivers.json to: {save_file}")
    with open(save_file, "w") as file:
        json.dump(state, file)

def load_selected_drivers():
    try:
        with open("sel_drivers.json", "r") as file:
            state = json.load(file)
        return state
    except FileNotFoundError:
        return {} 
    except json.JSONDecodeError:
        print("Error decoding JSON!")
        return {}

def real_close_window():
    messagebox.showwarning("Hint", "Please choose one of the modules")

def find_folder(driver_name, full_path):
    keywords = driver_search_keywords.get(driver_name, [driver_name.lower()])
    for item in os.listdir(full_path):
        item_path = os.path.join(full_path, item)
        #print(item_path)
        if any(keyword in item.lower() for keyword in keywords):
            return item_path
    return None

def install_driver(driver_path, driver_name):
    silent_drivers = ["KSC20", "Core-Temp", "bitwindows"]
    special_driver = ["Installer.exe"]
    
    if os.path.isfile(driver_path):
        if driver_name in silent_drivers:
            command = [driver_path, "/SILENT"]
        elif driver_name in special_driver:
            command = [driver_path, "-s", "-b"]
        else:
            command = [driver_path, "-s"]
        #print(f"Running: {' '.join(command)}")
        #command = [driver_path, "/SILENT"] if driver_name in silent_drivers else [driver_path, "-s"]
        subprocess.run(command, check=True)
    elif os.path.isdir(driver_path):
        for item in os.listdir(driver_path):
            item_path = os.path.join(driver_path, item)
            if item.lower().endswith(".exe"):
                if item in special_driver:
                    command = [item_path, "-s", "-b"]
                else:
                    command = [item_path, "/SILENT"] if driver_name in silent_drivers else [item_path, "-s"]
                #print(f"Running: {' '.join(command)}")
                subprocess.run(command, check=True)
                break
            elif item.lower().endswith(".inf"):
                #print(f"Installing INF driver: {item_path}")
                subprocess.run(["pnputil", "/add-driver", item_path, "/install"], check=True)
                break
    
def check_device_manager_for_issues():
    wmi_service = wmi.WMI()
    problem_device = []

    for device in wmi_service.Win32_PnPEntity():
        if device.ConfigManagerErrorCode and device.ConfigManagerErrorCode != 0:
            problem_device.append(f"{device.Name} - Error Code: {device.ConfigManagerErrorCode}")
    if problem_device:
        messagebox.showwarning("Device Manager Issues", "The following devices have issues:\n" + "\n".join(problem_device))
    else:
        messagebox.showinfo("Device Manager", "All devices are working properly with no issues")
            
def open_driver_window():
    root.withdraw()
    selected = selected_option.get()
    full_path = load_path()

    driver_window = ctk.CTkToplevel(root)
    driver_window.title("Drivers Selection")
    driver_window.geometry('600x600')
    driver_window.configure(fg_color='#cce6ff')
    driver_window.overrideredirect(True)

    def real_close_window2():
        driver_window.destroy()
        root.quit()

    second_bar = ctk.CTkFrame(driver_window, height=20, fg_color="#003359")
    second_bar.pack(fill="x", side="top")
    
    second_label = ctk.CTkLabel(second_bar, text="Drivers Selection", font=font2, text_color="white")
    second_label.pack(side="left", padx=10)

    second_bar.bind("<Button-1>", lambda event: start_drag(event, driver_window))
    second_bar.bind("<B1-Motion>", lambda event: on_drag(event, driver_window))

    driver_custom_exit_button = ctk.CTkButton(second_bar, text="X", command=real_close_window2, width=30, height=30, corner_radius=15, fg_color="red")
    driver_custom_exit_button.pack(side="right", padx=5)

    driver_window.after(200, lambda : driver_window.iconbitmap('Kontron.ico'))

    checkbox_frame = ctk.CTkFrame(driver_window, fg_color="#edf2f2", corner_radius=10)
    checkbox_frame.pack(padx=15, side="left")

    version_display_frame = ctk.CTkFrame(driver_window, fg_color="#f0f0f0", corner_radius=10, width=350)
    version_display_frame.pack(side="right", fill="both", expand=False, padx=(0, 20), pady=20) 
    
    version_title = ctk.CTkLabel(version_display_frame, text="Drivers Version", font=font1, text_color="black")
    version_title.pack(anchor="n", pady=(10, 5))
    
    version_text = ctk.CTkTextbox(version_display_frame, width=320, height=400, font=font)
    version_text.pack(fill="both", expand=True, padx=10, pady=10) 

    driver_options = ["Intel® Graphics", "Intel® Chipset", "Intel® Management Engine",  
                      "Intel® Serial IO", "Intel® Network", "Intel® GNA", "Intel® HID", 
                      "AcpiSmbus", "KSC20", "Core-Temp", "bitwindows", "FUSBCAN"]
    selected_drivers = {opt: ctk.IntVar(value=0) for opt in driver_options}

    if os.path.exists("sel_drivers.json"):
        state = load_selected_drivers()
        if state:
            for option, var in selected_drivers.items():
                if option in state and state[option] == 1:
                    var.set(1)

    for row, option in enumerate(driver_options):
        checkbox = ctk.CTkCheckBox(
            checkbox_frame, text=option, font=font,
            variable=selected_drivers[option],
            fg_color="#175e22", hover_color="#5a8a96",
            border_color="#3a728a", corner_radius=6
        )
        checkbox.pack(anchor="w", pady=5)
  
    def check_driver_version():
        w = wmi.WMI()
        result_text = ""
        keywords = ["Graphics","Chipset","Kontron","KSC","Ethernet","Management Engine","GNA","Serial IO","Core Temp","BurnInTest"]
    
        for device in w.Win32_PnPSignedDriver():
            if device.DeviceName:
                for keyword in keywords:
                    if keyword in device.DeviceName:
                        result_text += f"Device: {device.DeviceName}\n"
                        result_text += f"Driver Version: {device.DriverVersion}\n"
                        result_text += "-" * 63 + "\n"
        version_text.insert("1.0", result_text if result_text else "No matching drivers found.")

    def start_installation():
        full_path = load_path()
        if not full_path:
            messagebox.showerror("Error", "Driver path not found. Please go back and select a driver path.")
            return
            
        for option, var in selected_drivers.items():
            if var.get() == 1:
                folder_or_file = find_folder(option, full_path)
                #messagebox.showinfo("Hint", f"Looking for {option} in {full_path}, found: {folder_or_file}")
                if folder_or_file:
                    try:
                        install_driver(folder_or_file, option)
                        messagebox.showinfo("Hint", f"Successfully installed {option}")
                        time.sleep(3)
                    except subprocess.CalledProcessError as e:
                        messagebox.showerror("Installation Error", f"Failed to install {option}: {e}")
                else:
                    messagebox.showwarning("Driver Not Found", f"No folder or file found for {option}")
        
        messagebox.showinfo("Installation Complete", "Selected drivers have been processed.")
        check_device_manager_for_issues()
        check_driver_version()
    
        os.remove("path.txt")
        os.remove("sel_driver.json")
    
    button = ctk.CTkButton(checkbox_frame, command=lambda: (save_selected_drivers(selected_drivers), start_installation()),
                           text="Install", font=font1, fg_color="#3a728a", hover_color="#79a6ad", text_color="white", 
                           corner_radius=10).pack(pady=10, padx=10)
    
root = ctk.CTk()
root.geometry('280x200')
root.iconbitmap('Kontron.ico')
root.configure(fg_color='#cce6ff')
root.overrideredirect(True)

title_bar = ctk.CTkFrame(root, height=20, fg_color="#003359")
title_bar.pack(fill="x", side="top")

title_label = ctk.CTkLabel(title_bar, text="Module Selection", font=font2, text_color="white")
title_label.pack(side="left", padx=10)

custom_exit_button = ctk.CTkButton(title_bar, text="X", command=real_close_window, width=30, height=30, corner_radius=15, fg_color="red")
custom_exit_button.pack(side="right", padx=5)

options = ["DRIVER-3.5-SBC-ADN", "DRIVER-2.5-SBC-ADN", "DRIVER-3.5-SBC-RPL"]
selected_option = ctk.StringVar()

for option in options:
    checkbox = ctk.CTkRadioButton(
        root, text=option, font=font, variable=selected_option,
        value=option, border_color="#003359", hover_color="#002d75",
        fg_color="#c1d6d0"
    )  
    checkbox.pack(anchor="center", pady=5)

def save_path():
    #save_file = os.path.join(os.path.expanduser("~\Desktop"), "path.txt")
    save_file = ("path.txt")
    selected = selected_option.get()

    messagebox.showinfo("Hint",f"Saving path to: {save_file}")

    try:
        with open(save_file, "w") as file:
            file.write(os.path.join(default_path, selected))
        messagebox.showinfo("Hint",f"Path saved successfully to {save_file}")
    except Exception as e:
        messagebox.showinfo("Hint",f"Error saving path: {e}")

def load_path():
    if os.path.exists("path.txt"):
        with open("path.txt", "r") as file:
            return file.read().strip()
    return None

ctk.CTkButton(root, text="Next", font=font1,command=lambda:(open_driver_window(),save_path()),
              fg_color="#3a728a", hover_color="#79a6ad", text_color="white", 
              corner_radius=10).pack(pady=(20, 10), anchor="center")

title_bar.bind("<Button-1>", lambda event: start_drag(event, root))
title_bar.bind("<B1-Motion>", lambda event: on_drag(event, root))

if os.path.exists("path.txt"):
    root.withdraw()
    open_driver_window()

root.mainloop()

{'Intel® Graphics': 0, 'Intel® Chipset': 0, 'Intel® Management Engine': 0, 'Intel® Serial IO': 0, 'Intel® Network': 0, 'Intel® GNA': 0, 'Intel® HID': 0, 'AcpiSmbus': 0, 'KSC20': 0, 'Core-Temp': 1, 'bitwindows': 1, 'FUSBCAN': 0}
Saving sel_drivers.json to: sel_drivers.json


Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\Users\Kontron\AppData\Local\Programs\Python\Python311\Lib\tkinter\__init__.py", line 1967, in __call__
    return self.func(*args)
           ^^^^^^^^^^^^^^^^
  File "C:\Users\Kontron\AppData\Local\Programs\Python\Python311\Lib\site-packages\customtkinter\windows\widgets\ctk_button.py", line 554, in _clicked
    self._command()
  File "C:\TEMP\ipykernel_6468\2129123342.py", line 219, in <lambda>
    button = ctk.CTkButton(checkbox_frame, command=lambda: (save_selected_drivers(selected_drivers), start_installation()),
                                                                                                     ^^^^^^^^^^^^^^^^^^^^
  File "C:\TEMP\ipykernel_6468\2129123342.py", line 200, in start_installation
    folder_or_file = find_folder(option, full_path)
                     ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\TEMP\ipykernel_6468\2129123342.py", line 65, in find_folder
    for item in os.listdi